In [1]:
import pandas as pd
import sys

sys.path.append("../")


In [2]:
df = pd.read_csv("../data/casestudy_data.csv")
df.head(3)


,visit_id,date_time,experience,success,zipcode,pro,repeat_visit,days_since_last_visit,new_movers,year_home_built,home_market_value,length_of_residence,net_worth,income,mkt_organic_product_purchasers_value,mkt_trend_env_focused_hh_value,high_end_shoppers_value,do_it_yourselfer_value,montrd_home_security_sys_own_value,mkt_green_product_purchasers_value
0,14167,5/31/19 10:36,version1,0,30080,0,0,NaN,0,1987.0,320.0,8.0,250000.0,100000.0,29.0,25.0,25.0,55.0,28.0,39.0
1,195581,5/31/19 10:37,version1,0,32081,1,0,NaN,0,2006.0,366.0,4.0,250000.0,125000.0,40.0,28.0,23.0,60.0,18.0,46.0
2,451142,5/31/19 10:37,version2,0,91124,0,0,NaN,0,1988.0,504.0,13.0,500000.0,125000.0,21.0,12.0,16.0,76.0,32.0,31.0


In [3]:
df.columns


Index(['visit_id', 'date_time', 'experience', 'success', 'zipcode', 'pro',
       'repeat_visit', 'days_since_last_visit', 'new_movers',
       'year_home_built', 'home_market_value', 'length_of_residence',
       'net_worth', 'income', 'mkt_organic_product_purchasers_value',
       'mkt_trend_env_focused_hh_value', 'high_end_shoppers_value',
       'do_it_yourselfer_value', 'montrd_home_security_sys_own_value',
       'mkt_green_product_purchasers_value'],
      dtype='object')

In [6]:
df = df.sort_values("date_time")
df

,visit_id,date_time,experience,success,zipcode,pro,repeat_visit,days_since_last_visit,new_movers,year_home_built,home_market_value,length_of_residence,net_worth,income,mkt_organic_product_purchasers_value,mkt_trend_env_focused_hh_value,high_end_shoppers_value,do_it_yourselfer_value,montrd_home_security_sys_own_value,mkt_green_product_purchasers_value
0,14167,5/31/19 10:36,version1,0,30080,0,0,NaN,0,1987.0,320.0,8.0,250000.0,100000.0,29.0,25.0,25.0,55.0,28.0,39.0
1,195581,5/31/19 10:37,version1,0,32081,1,0,NaN,0,2006.0,366.0,4.0,250000.0,125000.0,40.0,28.0,23.0,60.0,18.0,46.0
2,451142,5/31/19 10:37,version2,0,91124,0,0,NaN,0,1988.0,504.0,13.0,500000.0,125000.0,21.0,12.0,16.0,76.0,32.0,31.0
3,278496,5/31/19 10:37,version1,0,60614,1,0,NaN,0,1968.0,433.0,8.0,500000.0,125000.0,37.0,10.0,12.0,80.0,40.0,37.0
4,316757,5/31/19 10:37,version2,0,76309,0,1,less than 7 days,0,1976.0,208.0,8.0,250000.0,100000.0,52.0,48.0,24.0,35.0,24.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110693,372625,6/9/19 9:59,version1,0,49684,0,1,NaN,0,1986.0,261.0,13.0,250000.0,75000.0,53.0,24.0,34.0,43.0,49.0,40.0
110694,332096,6/9/19 9:59,version2,0,48307,0,0,NaN,0,1984.0,351.0,13.0,500000.0,125000.0,43.0,32.0,26.0,42.0,26.0,62.0
110695,214309,6/9/19 9:59,version1,1,89129,0,1,NaN,0,1996.0,300.0,8.0,250000.0,100000.0,21.0,34.0,37.0,42.0,20.0,38.0
110696,223691,6/9/19 9:59,version1,0,30215,0,1,less than 1 day,1,1994.0,317.0,13.0,250000.0,100000.0,35.0,50.0,25.0,38.0,16.0,56.0


In [10]:
df["version1_took"] = df.experience.apply(func=lambda x: int(x == "version1"))
df["version2_took"] = df.experience.apply(func=lambda x: int(x == "version2"))

df["version1_succeed"] = df.apply(
    func=(lambda row: int(row.experience == "version1" and row.success == 1)), axis=1
)
df["version2_succeed"] = df.apply(
    func=(lambda row: int(row.experience == "version2" and row.success == 1)), axis=1
)




In [ ]:
df_mab = df[["date_time", "version1_took", "version2_took", "version1_succeed", "version2_succeed"]]